### NN Baseline (MNIST)

### Imports

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Create Dataset

In [27]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

### Split dataset

In [28]:
train_data, val_data = torch.utils.data.random_split(train_dataset, [50000, 10000])

### Create Dataloader

In [29]:
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Neural Network

In [30]:
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.flatten = nn.Flatten()
        
        self.fc = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)


### Training and Testing
- Training: 
- model.train() - Переводит модель в режим обучения
    - Dropout — включается (начинает случайно "обнулять" нейроны)
    - BatchNorm — использует текущие батч-статистики, а не сохранённые

- for X, y in loop:
    - Мы итерируемся по DataLoader, который на каждой итерации возвращает:
    - X — батч входных данных (например, картинки)
    - y — соответствующие истинные метки (labels)

-  X, y = X.to(device), y.to(device)
    - Отправляем данные на устройство

- optimizer.zero_grad()
    - Сбрасывает (обнуляет) градиенты всех параметров модели
    - Это нужно, потому что PyTorch по умолчанию накапливает градиенты после каждого loss.backward()
    - Если не обнулять — градиенты будут суммироваться, и веса начнут обновляться неправильно

- out = model(X)
    - Прямой проход (forward pass)
    - Модель возвращает логиты — это просто сырые выходы линейного слоя, до применения какой-либо нормализации

- loss = criterion(out, y)
    - Вычислениет функции потерь (loss)
    - loss — тензор, у которого requires_grad=True (если вычислен с параметров модели), поэтому при loss.backward() градиенты протекут к параметрам

- loss.backward() - шаг обратного распространения ошибки
    - вычисляет производные (градиенты) для каждого параметра модели
    - сохраняет эти значения в param.grad (тот же размер, что и сам параметр)
    - не обновляет параметры — только считает градиенты

- optimizer.step() - шаг оптимизации
    - После того как .backward() посчитает все градиенты, просто обновляет параметры модели на основе этих градиентов.
    



- Testing: 
- model.eval()
    - переводит модель в режим оценки/инференса
    - отключает dropout, BatchNorm использует сохранённые running статистики вместо батчевых

- with torch.no_grad():
    - контекст, внутри которого операции не создают вычислительный граф и requires_grad не будут включены
    - Экономит память и ускоряет вычисления при инференсе.

In [ ]:
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct = 0, 0
    
    loop = tqdm(loader, desc="Training", leave=False)
    
    for X, y in loop:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()     # 1️ очистить градиенты
        out = model(X)            # 2️ прямой проход
        loss = criterion(out, y)  # 3️ считаем ошибку
        loss.backward()           # 4️ считаем градиенты d(loss)/d(param)
        optimizer.step()          # 5️ обновляем параметры
        
        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        
        loop.set_postfix(loss=total_loss / (loop.n + 1), acc=correct / ((loop.n + 1) * loader.batch_size))
    
    return total_loss / len(loader), correct / len(loader.dataset)


def validate(model, loader, criterion):
    model.eval()
    total_loss, correct = 0, 0
    
    loop = tqdm(loader, desc="Validation", leave=False)
    
    with torch.no_grad():
        for X, y in loop:
            X, y = X.to(device), y.to(device)
            out = model(X)
            loss = criterion(out, y)
            
            total_loss += loss.item()
            correct += (out.argmax(1) == y).sum().item()
            
            loop.set_postfix(loss=total_loss / (loop.n + 1), acc=correct / ((loop.n + 1) * loader.batch_size))
    
    return total_loss / len(loader), correct / len(loader.dataset)


### Train

In [32]:
epochs = 5
for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
          f"Val loss: {val_loss:.4f}, acc: {val_acc:.4f}")



Epoch 1/5 | Train loss: 0.5676, acc: 0.8449 | Val loss: 0.3025, acc: 0.9110


Epoch 2/5 | Train loss: 0.2766, acc: 0.9193 | Val loss: 0.2436, acc: 0.9272


Epoch 3/5 | Train loss: 0.2243, acc: 0.9338 | Val loss: 0.2091, acc: 0.9383


Epoch 4/5 | Train loss: 0.1852, acc: 0.9460 | Val loss: 0.1989, acc: 0.9401


Epoch 5/5 | Train loss: 0.1575, acc: 0.9534 | Val loss: 0.1559, acc: 0.9522


Что делает Flatten, Linear, Conv2d, Dropout, BatchNor(слои)
vanishing / exploding gradients
Initialization